In [4]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List
from tqdm import tqdm
import os
import json
import random
from numpy import longdouble
from sklearn.utils import shuffle
# import pathos.multiprocessing as mpathos
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')

text = "This is a sample sentence to tokenize."
tokens = sp.encode_as_pieces(text)
print(tokens)
print(sp.encode_as_ids(text))
print(sp.decode(sp.encode_as_ids(text)))

['▁This', '▁is', '▁a', '▁sa', 'mp', 'le', '▁sentence', '▁to', '▁to', 'ken', 'ize', '.']
[102, 22, 14, 1693, 2265, 421, 7627, 6, 6, 4296, 3055, 3]
This is a sample sentence to tokenize.


In [5]:
def find_max_sequence_length():
    maxlen = 0
    prefix = './PhishingEmails/'  # Adjust this to your file path
    
    for file in tqdm(os.listdir(prefix)):
        with open(prefix + file) as jsonFile:
            jsonStr = jsonFile.read()
            email_dict = json.loads(jsonStr)
            setupData = sp.encode_as_ids(
                email_dict['email_subject'].strip() + ". " + email_dict['email_body'].strip()
            )

            if(len(setupData) < 801):
                maxlen = max(maxlen, len(setupData))

    return maxlen

In [6]:
def load_emails(batch_size, windows_size):

    max_sequence_length = find_max_sequence_length()
    dataset = []
    batch_counter = 0
    batch = []
    ctr = 0
    
    prefix = './PhishingEmails/' #change this to the prefile thing such as './celebA'

    for file in tqdm(os.listdir(prefix)):
        
        with open(prefix + file) as jsonFile:
            jsonStr = jsonFile.read() #json file as a string
            email_dict = json.loads(jsonStr) #converts to dictionary
        
        setupData = sp.encode_as_ids(email_dict['email_subject'].strip() + ". " + email_dict['email_body'].strip())
        pad_length = max_sequence_length - len(setupData)

        if(len(setupData) <= max_sequence_length):
            # print(setupData,"\n\n\n")
            ctr+=1

            for i in range(windows_size, len(setupData) - windows_size - 1):

                if batch_counter < batch_size:
                    batch.append(np.array(setupData[i:i+windows_size],dtype=longdouble))
                    batch_counter += 1
                else:
                    dataset.append(batch)
                    batch = []
                    batch_counter = 0
    dataset = np.array(dataset)
    np.random.shuffle(dataset)
    # labels = np.array(labels)

    
    return dataset

In [18]:
import multiprocessing
from train_LSTM import train

def grid_search(input_size):

    for BS in [4,8,16,32,64]:

        dataset = load_emails(BS, input_size)

        for LR in [0.1,0.01,0.001,0.0001,0.00001]:

            for HS in [64,128,192]:

                for norm in [0.1,0.5,0.85,1.1,2]:
                     
                    multiprocessing.Process(target=train, args=(dataset,input_size, HS, BS, LR )).start()
                    



ImportError: cannot import name 'train' from 'train_LSTM' (c:\Users\josep\Documents\GitHub\CarpLSTM\train_LSTM.py)

In [8]:


batch_size = 16
input_size = 50
# windows_size = 50

hidden_size = 128
# + hidden_size



# dataset = load_emails(batch_size, input_size)

In [17]:
grid_search(input_size)

100%|██████████| 3332/3332 [00:18<00:00, 176.06it/s]


AttributeError: module 'train_LSTM' has no attribute 'train'

In [413]:

# ex_email = dataset[random.randint(0, 50)]

finalCell, losses = train_LSTM(dataset, input_size, hidden_size, batch_size, 0.001)

plt.plot(np.arange(len(losses)) * batch_size, losses)
plt.title("training curve")
plt.xlabel("number of emails trained on")
plt.ylabel("loss")
plt.show()

C:\Users\josep\AppData\Local\Temp\ipykernel_12292\4162846258.py:3: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-1*x))


4413.348611453071
4410.103651372504
4408.478297722507
4411.959116240481
4408.452369775776
4411.149946906217
4406.745966426501
4409.037610148325
4408.012365238779
4407.395042886838
4407.862809819659
4406.954442522369
4409.173823896901
4403.923276488455
4411.063434520147
4406.224961132432
4406.207402763207
4412.648880828224
4408.228731342289
4410.169101075328
4410.104089156578
4407.266483054641
4404.242670526743
4406.426003187048
4405.353764751575
4405.250245494225
4407.7275950572
4406.448738628031
4402.378353323294
4407.339242255992
4407.350991293392
4405.742965636011
4404.7771589606955
4411.892128552836
4405.799249469802
4407.031367927125
4401.439941876885
4407.535610833457
4403.533781472305
4407.549234845564
4404.923458955691
4407.264660028071
4405.292138798357
4404.244174044322
4405.149375336207
4404.696893565161
4402.289348787429
4402.268599289424
4400.537063495407
4397.268107445006
4400.563757811523
4410.206036617035
4407.584584384337
4404.647730131657
4405.510103806087
4404.118691